In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from scipy.optimize import curve_fit

In [2]:
exp = get_exp((39,269,329),directory_project)


begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00


In [59]:
def criter(max_growth,length):
    return(a*length+b*max_growth>=2.5 and max_growth>=50)

In [3]:
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)

failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure 

In [17]:
all_max_speeds += max_speeds

In [12]:
len(all_max_speeds)

10672

In [5]:
all_max_speeds = []

In [18]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist([speed for speed in all_max_speeds if speed<400 and speed>150],30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([53., 52., 61., 56., 53., 47., 38., 44., 43., 40., 37., 30., 34.,
        31., 35., 27., 35., 29., 35., 39., 24., 22., 17.,  6.,  8.,  4.,
         4.,  2.,  3.,  3.]),
 array([150.40605602, 158.11690353, 165.82775104, 173.53859855,
        181.24944606, 188.96029358, 196.67114109, 204.3819886 ,
        212.09283611, 219.80368362, 227.51453113, 235.22537864,
        242.93622615, 250.64707366, 258.35792117, 266.06876869,
        273.7796162 , 281.49046371, 289.20131122, 296.91215873,
        304.62300624, 312.33385375, 320.04470126, 327.75554877,
        335.46639628, 343.17724379, 350.88809131, 358.59893882,
        366.30978633, 374.02063384, 381.73148135]),
 <a list of 30 Patch objects>)

In [6]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
x = np.linspace(0,12000)
ax = fig.add_subplot(111)
ax.scatter(total_growths,max_speeds)
a= 0.0005
b= 0.01
thresh = 2
ax.plot(x,2.5/b-a/b*x)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
len(RH)

230

In [61]:
hyph_anas_tip_hyph = [hyphat for hyphat in exp.hyphaes if len(hyphat.ts)>=2 and hyphat.end.degree(hyphat.ts[-1])>=3 and hyphat.end.degree(hyphat.ts[-2])>=3]


In [62]:
chosen_RH = [rh for rh in RH if rh.ts[-1]==60]#+[rh for rh in BAS if len(rh.ts)>0 and rh.ts[-1]==60]

In [63]:
rh_not_anas = [rh for rh in chosen_RH if rh not in hyph_anas_tip_hyph]
rh_anas = [rh for rh in chosen_RH if rh in hyph_anas_tip_hyph]

In [64]:
plt.close('all')
rh_an = choice(rh_anas)
plot_raw_plus(exp,rh_an.ts[-1],[rh_an.root.label,rh_an.end.label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
stopping = []
for rh in rh_not_anas:
    if len(rh.ts)>2:
        t=rh.ts[-2]
        tp1= rh.ts[-1]
        pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
        lapse_time = get_time(exp, t, tp1)
        curv_growth = np.sum([get_length_um(seg) for seg in pixels])
        speed = curv_growth/lapse_time
        if speed<10:
            stopping.append(rh)


failure in finding closest edge


In [25]:
len(stopping), len(rh_anas),len(chosen_RH),len(rh_not_stopping_or_anastomosing)

(22, 44, 126, 60)

In [66]:
lapses_stopping= []
for rh in stopping:
    for index, t in enumerate(rh.ts[:-1]):
        tp1= rh.ts[index+1]
        pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
        lapse_time = get_time(exp, t, tp1)
        curv_growth = np.sum([get_length_um(seg) for seg in pixels])
        speed = curv_growth/lapse_time
        if speed<10:
            lapses_stopping.append(get_time(exp,rh.ts[0],t))
            break
        


failure in finding closest edge


In [67]:
lapse_anas = []
for hyph in hyph_anas_tip_hyph:
    if hyph in RH:
        for t in hyph.ts:
            if hyph.end.degree(t)>=3:
                time = get_time(exp,hyph.ts[0],t)
                lapse_anas.append((hyph.ts[0],time))
                break
lapses_anas = [lapse for t,lapse in lapse_anas]

In [68]:
rh_not_stopping_or_anastomosing = [rh for rh in chosen_RH if rh not in stopping and rh not in rh_anas]

In [69]:
lapses_continuing = [get_time(exp,rh.ts[0],rh.ts[-1]) for rh in rh_not_stopping_or_anastomosing]

In [70]:
counts = []
lapses = range (exp.ts)
for lapse in lapses:
    counting=0
    for liste in [lapses_anas,lapses_stopping]:
        counting+=len([x for x in liste if x>=lapse])
    counts.append(counting)

In [18]:
len(RH)

112

In [79]:
fig = plt.figure()
ax = fig.add_subplot(111)


def func2(t,alpha):
    return((np.exp(-alpha*(t))))
popt1, pcov = curve_fit(func2,lapses[0:],np.array(counts)[0:]/counts[0],bounds = ([0],1*[np.inf]),p0=[1])
# popt2, pcov = curve_fit(func2, times, total_anastomosis_theory,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])

ax.scatter(lapses,np.array(counts)/counts[0], label = 'Observed proportion of hyphae not anastomosing')
ax.plot(lapses,func2(np.array(lapses),*popt1),label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='blue')
ax.set_xlabel('lapse (h)',size=20)
ax.set_ylabel('P(t>lapse)',size =20)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
3.6*1000/np.log(2)/24

216.40425613334455

In [46]:
plt.close('all')
plot_raw_plus(exp,-1,[rh.root.label for rh in RH])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plt.close('all')
plot_raw_plus(exp,-1,[2646, 9824])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
a = [79,11371,12]
b = [79,5749,6]
c = [2646, 9824,7]
d = [2271, 6297,3]
e = [2646,7628,4]
f = [430,7503,10]

In [22]:
x=a
list_node = nx.shortest_path(
            exp.nx_graph[-1],
            source=x[0],
            target=x[1],
            weight="weight",
        )
get_length_um_node_list(list_node,exp,-1)/x[2]

1553.1546677321942

In [25]:
frequs = []
for trup in [a,b,c,d,e,f]:
    x = trup
    list_node = nx.shortest_path(
            exp.nx_graph[-1],
            source=x[0],
            target=x[1],
            weight="weight",
        )
    frequs.append(get_length_um_node_list(list_node,exp,-1)/x[2])

In [26]:
np.mean(frequs)

1561.5828533268539

In [34]:
(2.84+1.10-1.03)*1500/np.log(2)/24

262.39016056168026

In [45]:
get_length_um_node_list(list_node,exp,-1)/x[2]

1875.891732006734

In [34]:
np.mean(lapses_stopping),np.mean(lapses_anas)

(10.04469696969697, 7.328112449799197)

In [47]:
len(lapses_stopping)

22

In [51]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(lapses_anas,10,density= True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([0.12037348, 0.07482676, 0.03253337, 0.00976001, 0.00976001,
        0.01301335, 0.        , 0.00650667, 0.        , 0.00325334]),
 array([ 0.98333333,  4.68666667,  8.39      , 12.09333333, 15.79666667,
        19.5       , 23.20333333, 26.90666667, 30.61      , 34.31333333,
        38.01666667]),
 <a list of 10 Patch objects>)

In [32]:
plt.close('all')
rh_an = choice(rh_not_stopping_or_anastomosing)
print(rh_an.ts)
plot_raw_plus(exp,rh_an.ts[-1],[rh_an.root.label,rh_an.end.label])

[49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.close('all')
rh_an = choice(stopping)
plot_raw_plus(exp,rh_an.ts[-1],[rh_an.root.label,rh_an.end.label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …